![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx to tune Meta `llama-2-13b-chat` model with Consumer Financial Protection Bureau document

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support of prompt tuning in watsonx.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to:
- upload of dataset with prompts to container (project)
- finding best tuning parameters (hyper-parameters optimization)
- trigger prompt tuning process
- review of prompt tuning process details/results
- deploy prompt-tuned model/asset
- evaluation (via inference) of prompt-tuned model/asset


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Data loading](#data_loading)
- [Initialize experiment](#initialize)
- [Hyper-parameters optimization](#hpo)
- [Run Prompt Tuning](#run_tuning)
- [Prompt Tuning details](#run_details)
- [Deploy](#deploy)
- [Foundation Models Inference](#models_inference)
- [Analyze the satisfaction](#predict)
- [Summary and next steps](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependencies

In [ ]:
!pip install ibm-watsonx-ai | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install matplotlib | tail -n 1
!pip install wget | tail -n 1
!pip install scikit-optimize | tail -n 1

### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [ ]:
import os
import getpass
from dotenv import load_dotenv

try:
    load_dotenv()
    api_key = os.getenv("API_KEY")
except Exception:
    api_key = getpass.getpass("Please enter your api key (hit enter): ")
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com"
project_id = os.getenv("PROJECT_ID")

if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

Create an instance of APIClient with authentication details.

In [ ]:
from ibm_watsonx_ai import APIClient

client = APIClient(creds)

To be able to interact with all resources available in Watson Machine Learning, you need to set **project_id** which you will be using.

In [ ]:
client.set.default_project(project_id)

<a id="data_loading"></a>
## Data loading

This example uses the Consumer Financial Protection Bureau training dataset. 

In [ ]:
filename = 'cfpb_train.json'

Download training data from git repository and create data assets.

In [ ]:
import wget
import os

url = "https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/prompt_tuning/cfpb_train.json"
if not os.path.isfile(filename): 
    wget.download(url)
    
asset_details = client.data_assets.create(name=filename, file_path=filename)
asset_id = client.data_assets.get_id(asset_details)

Define connection information to training data.

In [ ]:
from ibm_watsonx_ai.helpers import DataConnection

data_conn = DataConnection(data_asset_id=asset_id)

<a id="initialize"></a>
## Initialize experiment

In [ ]:
from ibm_watsonx_ai.experiment import TuneExperiment

experiment = TuneExperiment(creds, project_id=project_id)

All avaliable tasks are presented under `Tasks` Enum

In [ ]:
{task.name: task.value for task in experiment.Tasks}

<a id="hpo"></a>
## Hyper-parameters optimization (optional, can take a while to complete!)

The goal of hyper-parameters optimization is to find the best `learning_rate` value yielding the lowest loss. The search space can be extended to other parameters like `num_epochs` or `batch_size`.
**Hint:** To execute the code change the cell type to Code and remove markdown marks. Note that HPO takes significant time and resources.


### Train function
Let's define the `train_evaluate` function that will accepts set of parameters from SEARCH dict and return the loss value.
In addition, we will measure the elapsed time. To speed up the HPO time we will use only 3 epochs.

In [ ]:
# import time

# def train_evaluate(search_params):
#     prompt_tuner = experiment.prompt_tuner(name="Prompt tuning with HPO - " + str(search_params),
#                                            base_model='meta-llama/llama-2-13b-chat',
#                                            task_id=experiment.Tasks.CLASSIFICATION,
#                                            tuning_type=experiment.PromptTuningTypes.PT,
#                                            num_epochs=3,
#                                            batch_size=8,
#                                            max_input_tokens=256,
#                                            max_output_tokens=20,
#                                            verbalizer="Including narratives choice the best match product with the items from the list: 'credit_card', 'debt_collection', 'mortgages_and_loans', 'credit_reporting', 'retail_banking'. Input: {{input}} Output: ",
#                                            auto_update_model=False,
#                                            **search_params
#                                            )

#     tuning_details = prompt_tuner.run(training_data_references=[data_conn], background_mode=True)
#     print(f"Starting the tune...")
#     print(f"hyper-parameters", str(search_params))
#     start = time.time()

#     while prompt_tuner.get_run_status() not in ['failed', 'canceled', 'completed']:
#         tuning_details = prompt_tuner.get_run_details()
#         time.sleep(10)
#     end = time.time() - start

#     if prompt_tuner.get_run_status() in ['failed', 'canceled']:
#         print("Model tuning failed or halted")
#         print(f"get_run_details: {prompt_tuner.get_run_details()}")
#         return 10

#     try:
#        loss = prompt_tuner.summary().loss.values[0]
#     except:
#         loss = 10

#     print(prompt_tuner.get_run_status(), 'loss: ',  loss, ' | run time', end)
#     return loss

### Search space and optimization
In next step let's define the search space for `learning_rate` and objective function.

In [ ]:
# import skopt

# SPACE = [
#     skopt.space.Real(0.001, 0.005, name='learning_rate', prior='log-uniform')
# ]

# @skopt.utils.use_named_args(SPACE)
# def objective(**params):
#     return train_evaluate(params)

# results = skopt.forest_minimize(objective, SPACE, n_calls=10)
# best_loss = results.fun
# best_params = results.x
# # best_id = results.id

# print('best result: ', best_loss)
# print('best parameters: ', best_params)

<a id="run_tuning"></a>
## Run Prompt Tuning 

**Note**: This will take approx. 20-25 mins, if you wish you can skip this step, navigate to **Foundation Models Inference on watsonx.ai** and use our tuned/deployed model with provided credentials.

Run a prompt tuning process of foundation model on top of the training data referenced by DataConnection (tuning may take some time).

Define `prompt_tuner` parameters. In the example below the `learning_rate` value was established using the HPO runs described above.

**Action:** For parameters details, see <a href="https://ibm.github.io/watsonx-ai-python-sdk/tune_experiment.html#ibm_watsonx_ai.experiment.fm_tune.TuneExperiment.prompt_tuner" target="_blank" rel="noopener no referrer">documentation</a>.

In [ ]:
prompt_tuner = experiment.prompt_tuner(name="Llama-2-13b-chat prompt tuning - python SDK",
                                       task_id=experiment.Tasks.CLASSIFICATION,
                                       base_model='google/flan-t5-xl',
                                       accumulate_steps=32,
                                       batch_size=8,
                                       learning_rate=0.05881198993298603,
                                       max_input_tokens=256,
                                       max_output_tokens=10,
                                       num_epochs=6,
                                       tuning_type=experiment.PromptTuningTypes.PT,
                                       verbalizer="Including narratives choice the best match product with the items from the list: 'credit_card', 'debt_collection', 'mortgages_and_loans', 'credit_reporting', 'retail_banking'. Input: {{input}} Output: ",
                                       auto_update_model=True
                                       )

You can review previous set parameters

In [ ]:
prompt_tuner.get_params()

By changing the `background_mode` parameter to `True`, the prompt tuning process will run in the background.

In [ ]:
import time
start = time.time()
tuning_details = prompt_tuner.run(training_data_references=[data_conn], background_mode=True)
print(f"Starting the tune...")
while prompt_tuner.get_run_status() not in ['failed', 'canceled', 'completed']:
    time.sleep(60)
    try:
        loss = prompt_tuner.summary().loss.values[0]
    except:
        loss = None
    print("Loss: ", loss)
end = time.time() - start
try:
    loss = prompt_tuner.summary().loss.values[0]
except:
    loss = 10
print(prompt_tuner.get_run_status(), ', Loss: ',  loss, ' | Run time: ', end)

**Note**: you can view your tuning experiments by navigating to your project on watsonx.ai user interface.

<a id="run_details"></a>
## Prompt Tuning details

Check status/state of initialized Prompt Tuning run if ran in background mode or when process finish if background mode is off.

In [ ]:
prompt_tuner.get_run_status()

Let's summarize the run.

In [ ]:
prompt_tuner.summary()

Plot learning curves.

In [ ]:
prompt_tuner.plot_learning_curve()

<a id="deploy"></a>
### Deploy

You can specify `model_id` from tuning details.

In [ ]:
model_id = None

if 'model_id' in tuning_details.get('entity', {}):
    model_id = tuning_details['entity']['model_id']
model_id

Create online deployment for published model.

In [ ]:
from datetime import datetime
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "PT DEPLOYMENT SDK - project",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.SERVING_NAME : f"pt_sdk_deployment_{datetime.utcnow().strftime('%Y_%m_%d_%H%M%S')}"
}
deployment_details = client.deployments.create(model_id, meta_props)

Additionally you can get deployment details by printing `deployment_details`

In [ ]:
deployment_details

You can specify `deployment_id` from deployment details.

In [ ]:
deployment_id = deployment_details['metadata']['id']
deployment_id

<a id="models_inference"></a>
## Foundation Models Inference on watsonx.ai

Provide a set of model parameters.

### Optionally, if you want to use an already Tuned (and deployed model), use the provided credentials. Else, go ahead and tune your own model, it will take a while (approx. 20-25 mins)

In [ ]:
creds = {
    'url': "https://us-south.ml.cloud.ibm.com",
    'api_key': getpass.getpass("Please enter the corresponding api key (hit enter): "),
    'project_id': input("Please enter the corresponding project ID (hit enter): "),
    'deployment_id': input("Input the deployment_id (hit enter): "),
}


In [ ]:
project_id = creds["project_id"]
deployment_id = creds["deployment_id"]

In [ ]:
client = APIClient(creds)
client.set.default_project(project_id)

In [ ]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

generate_params = {
    GenParams.MAX_NEW_TOKENS: 20,
    GenParams.STOP_SEQUENCES: ["\n\n"]
}

Initialize the `ModelInference` class.

In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference

tuned_model = ModelInference(
    deployment_id=deployment_id,
    params=generate_params,
    api_client=client
)

Alternatively you can use `credentials` and `project_id` to initialize the `ModelInference` class.

```
tuned_model = ModelInference(
    deployment_id=deployment_id,
    params=generate_params,
    credentials=credentials,
    project_id=project_id
)

```

Get deployment model inference details.

In [ ]:
tuned_model.get_details()

Analyze the product class for a sample prompt.

In [ ]:
response = tuned_model.generate_text(prompt="Including narratives choice the best match product with the items from the list: 'credit_card', 'debt_collection', 'mortgages_and_loans', 'credit_reporting', 'retail_banking'.\nComment: hi landed job reside ca needed room rent found place rent paid deposit dollar however position going didnt work longer needed rent place bay asked landlord refund security deposit refused told called back wellsfargo disputed transaction recently noticed card reversal checking account got charged amount dollar called bank werent able refund money also emailed landlord asking refund money ten day passed still response hope cfpb take action successfully resolve issue thank\nProduct:\n")
response

<a id="predict"></a>
## Analyze the product classes.

Download and prepare the `cfpb_test` dataset.

In [ ]:
import pandas as pd

filename = 'cfpb_test.json'
url = "https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/prompt_tuning/cfpb_test.json"
if not os.path.isfile(filename):
    wget.download(url)
    
data = pd.read_json(filename)

In [ ]:
prompts = list(data.input)
products = list(data.output)

In [ ]:
prompts_batch = ["\n".join([prompt]) for prompt in prompts]

### Calculate the accuracy of tuned model

In [ ]:
tuned_model_results = tuned_model.generate_text(prompt=prompts_batch)

Let's strip whitespaces from the generated text to have reliable accuracy calculated.

In [ ]:
tuned_model_results = [x.strip() for x in tuned_model_results]

In [ ]:
from sklearn.metrics import accuracy_score

print(f'accuracy_score: {accuracy_score(products, tuned_model_results)}')

### Calculate the accuracy of base model

Initialize the `ModelInference` class with base model.

**Note**: if you have tuned your own model, you need to make sure the `model_id` below matches the base model you have used for tuning.

In [ ]:
base_model = ModelInference(
    model_id='meta-llama/llama-2-13b-chat',
    params=generate_params,
    api_client=client
)

In [ ]:
base_model_results = base_model.generate_text(prompt=prompts_batch)

Let's strip whitespaces from the generated text to have reliable accuracy calculated.

In [ ]:
base_model_results = [x.strip() for x in base_model_results]

In [ ]:
from sklearn.metrics import accuracy_score

print(f'base model accuracy_score: {accuracy_score(products, base_model_results)}')

In [ ]:
print(base_model_results)

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use prompt tuning in watsonx for analyze the product classes for a sample prompts.
 
Check out our _<a href="https://ibm.github.io/watson-machine-learning-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors: 

**Lukasz Cmielowski**, Senior Technical Staff Member at Watson Machine Learning,

**Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.